# 畳み込みニューラルネットワーク

## 畳み込み層

- 畳み込み層: CNN（Convolutional Neural Network）で新たに取り入れられる層の一つ。
- 特徴マップ(feature map): 畳み込み層における入出力データのこと
- 入力特徴マップ(input feature map): 畳み込み層における入力データのこと
- 出力特徴マップ(output feature map): 畳み込み層における出力データのこと

http://www.hpc.co.jp/AboutDeepLearning.html
![http://www.hpc.co.jp/images/DL_kaisetsu_09.png](http://www.hpc.co.jp/images/DL_kaisetsu_09.png)

http://pythonskywalker.hatenablog.com/entry/2016/12/26/164545  
![https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226141944.png](https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226141944.png)

### 全結合層の問題点

これまでのニューラルネットワークは隣接する層のすべてのニューロンとニューロンの間で結合がある（fully-connected; 全結合;）状態だった。  
我々はこれまでそれぞれの結合部をAffineレイヤ（行列の内積計算）で実装しReLUにて発火させた。これは何の考慮もせずに入力値をそのまま行列に変換してニューラルネットワークを構築したことになる。つまり今回のMNISTの手書き文字の場合、元々あった縦・横などの「データの形状情報」やRGB（ピクセルの色合いの順序情報;R要素・G要素・B要素の区別;）が失われてしまうことになる。  
__データをそのまま全結合層の入力（行列）として扱った場合3次元以上の情報は失われてしまう！__  
例えば、画像を行列として扱った場合に失われる情報としては以下が考えられる
- ピクセルのR,G,B,Aのそれぞれの濃度
- 縦・横のピクセル幅
- 上下左右で連続したピクセル間の値の推移

### 畳み込み演算

畳み込み演算は画像処理で言う「フィルター演算」にあたる。  
http://www.hpc.co.jp/AboutDeepLearning.html  
![http://www.hpc.co.jp/images/DL_kaisetsu_10.png](http://www.hpc.co.jp/images/DL_kaisetsu_10.png)

「フィルター」はカーネルとも呼ばれ、入力データに対してあるウインドウサイズの区画で積和演算を行う。全結合のニューラルネットワークでは行列が重みパラメータとなっていたが、CNNの場合はこのフィルターの乗算内容が「重み」に対応する。  
積和演算の結果は各ウィンドウの位置ごとに取得され、結果は新たな出力データとなる。  
その出力データの行列にスカラをスカラ加算（行列の要素全てに等しく加算）することでバイアスも実現する。  
バイアスはスカラなので、1 x 1の行列となる。  

### パディング

入力データの大外にダミーの枠を用意することで、積和演算結果のサイズを調整できる。  
畳み込み演算を繰り返すと段々出力サイズが小さくなるので、最終的な出力サイズが1になることを防ぐことができる。  
畳み込み演算回数（隠れ層の数）によっては不要の場合がある。

### ストライド

フィルターの適用する位置の間隔のこと。積和演算が一回終わった時点でウィンドウがスライドする幅。  
パディングとは逆に、ストライドは大きくすると積和演算結果の出力サイズは小さくなる。

#### ストライドと出力サイズの関係

- 出力サイズ ($OH = \frac{H+2P-FH}{S} + 1, OW = \frac{W+2P-FW}{S} + 1$) 
    - 入力サイズ $(H, W)$
    - フィルターサイズ $(FH, FW)$
    - パディング $P$
    - ストライド $S$
  
ただし、ストライドで割り切れない値の場合は長方形の形にならない（行列において欠損データが発生する）為
割り切れない値を丸めるかダミー値を挿入すること。

### 3次元データの畳み込み演算

通常の畳込み演算にチャンネル（奥行き）を加えたもの。  
- 適用フィルターの枚数（=重み） x 入力データ = 出力チャンネル数となる。
- バイアスは（フィルター数, 1, 1）という形でブロードキャスト演算により適用される。

http://pythonskywalker.hatenablog.com/entry/2016/12/26/164545  
![https://camo.qiitausercontent.com/24262ae92d579768024ae0a7a1d00271d66e8917/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f32353939302f66333461363337322d393730632d636365382d353861642d3139316432373566656635372e706e67](https://camo.qiitausercontent.com/24262ae92d579768024ae0a7a1d00271d66e8917/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f32353939302f66333461363337322d393730632d636365382d353861642d3139316432373566656635372e706e67)

### 畳み込み演算のバッチ演算

単にN個のデータを重ねただけ。（4次元データの演算）

http://pythonskywalker.hatenablog.com/entry/2016/12/26/164545  
![https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226164341.png](https://cdn-ak.f.st-hatena.com/images/fotolife/r/riikku/20161226/20161226164341.png)

## プーリング層

プーリング層(Pooling)は引数フィルターなどを持たずに、関数のみで入力を縦・横の空間を狭くして出力する演算のこと。  
一般的にはウィンドウサイズ＝ストライドとなるように設定する。  
MAXプーリングは以下のような形で演算がなされる。他にも、Averageプーリングなどがある。

https://indoml.com/2018/03/07/student-notes-convolutional-neural-networks-cnn-introduction/  
![https://indoml.files.wordpress.com/2018/03/pooling-layer3.png?w=648&h=180)](https://indoml.files.wordpress.com/2018/03/pooling-layer3.png?w=648&h=180)

### プーリング層の特徴

#### 学習するパラメータが無い

プーリング層は畳み込みそうと違い学習パラメータを持たない。（層自体が成長しない）

##### チャンネル層は変化しない

プーリング層を通過しても入力データと出力データのチャンネル数は変化しない。

##### 微小な位置変化に対してロバスト（頑健）

入力データの小さなズレに対してもプーリング層は同じような結果を返す。  
入力データが多少ズレていてもウィンドウサイズに収まっている程度のズレであれば吸収できる。

## Convolution／Poolingレイヤの実装